<a href="https://colab.research.google.com/github/JuanQuiroga12/DeepLearning/blob/main/ModeloMejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
# Generador Automático de Preguntas con GPT-Neo (ENFOQUE MANUAL)
# Universidad Militar Nueva Granada
# Deep Learning - Homework 7

Este notebook implementa un modelo GPT-Neo para generar preguntas de opción múltiple
a partir de textos educativos en español, utilizando un enfoque manual de carga de dataset
para mayor robustez.

Trabajo de: Juan Quiroga y Marielby Paz
"""

# Importamos las librerías necesarias
import os
import numpy as np
import pandas as pd
import torch
import random
import json
import re
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import textwrap
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler,
    AutoConfig,
    BitsAndBytesConfig
)
import datasets
from datasets import Dataset, DatasetDict
import wandb
import gc
import time

# Configuración inicial
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "EleutherAI/gpt-neo-1.3B"  # Modelo más pequeño
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 5e-5
NUM_EPOCHS = 15
MAX_LENGTH = 512
OUTPUT_DIR = "gpt-neo-question-generator"
USE_8BIT = True  # Desactivamos 8-bit para evitar problemas

# Establecer semillas para reproducibilidad
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Crear el directorio de salida si no existe
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print(f"Utilizando device: {DEVICE}")

# INSTRUCCIONES PARA DESCARGAR MANUALMENTE XQUAD-ES
print("""
===== INSTRUCCIONES PARA DESCARGAR DATOS =====

1. Descarga XQuAD-ES desde GitHub:
   - Ve a: https://github.com/deepmind/xquad
   - Descarga el archivo 'xquad.es.json' desde la carpeta 'data'

2. Sube el archivo a tu sesión de Colab:
   - Usa el menú a la izquierda -> Archivos -> Subir
   - Selecciona el archivo xquad.es.json que descargaste

3. Ejecuta este notebook después de subir el archivo

Si tienes problemas para descargar XQuAD, puedes continuar con el dataset sintético.
""")

# Clase para el dataset de preguntas de XQuAD
class XQuADDataset(Dataset):
    def __init__(self, file_path=None, use_synthetic=False, tokenizer=None, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.examples = []

        if file_path and os.path.exists(file_path) and not use_synthetic:
            print(f"Cargando XQuAD desde {file_path}...")
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # XQuAD tiene una estructura específica: {'data': [{'paragraphs': [{'context': '...', 'qas': [{'question': '...', 'answers': ...}]}]}]}
                for article in data['data']:
                    for paragraph in article['paragraphs']:
                        context = paragraph['context']
                        for qa in paragraph['qas']:
                            question = qa['question']
                            answers = qa['answers']
                            if answers and len(answers) > 0:
                                # Tomamos la primera respuesta como la correcta
                                first_answer = answers[0]
                                if isinstance(first_answer, dict) and 'text' in first_answer:
                                    answer_text = first_answer['text']
                                    self.examples.append({
                                        'context': context,
                                        'question': question,
                                        'answer': answer_text
                                    })

                print(f"Cargados {len(self.examples)} ejemplos de XQuAD")
            except Exception as e:
                print(f"Error al cargar XQuAD: {e}")
                use_synthetic = True
        else:
            use_synthetic = True

        if use_synthetic:
            print("Usando dataset sintético...")
            self.examples = self.create_synthetic_dataset()
            print(f"Creados {len(self.examples)} ejemplos sintéticos")

    def create_synthetic_dataset(self):
        # Lista de contextos educativos en español
        contexts = [
            "La fotosíntesis es un proceso químico que convierte el dióxido de carbono y el agua en oxígeno y glucosa usando la luz solar. Este proceso es fundamental para la mayoría de los seres vivos en la Tierra, ya que es la principal forma en que la energía del sol se incorpora a las cadenas alimentarias. La clorofila, un pigmento verde presente en las plantas, algas y algunas bacterias, juega un papel crucial en este proceso al capturar la energía luminosa.",

            "El sistema solar está compuesto por el Sol y todos los objetos que orbitan a su alrededor, incluyendo ocho planetas principales: Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno. También incluye planetas enanos como Plutón, numerosos satélites naturales como la Luna, y miles de asteroides, cometas y meteoroides. El Sol contiene más del 99% de la masa del sistema solar y proporciona la energía que sostiene la vida en la Tierra.",

            "La Revolución Industrial fue un periodo de profunda transformación económica, social y tecnológica que comenzó en Gran Bretaña a finales del siglo XVIII y se extendió por Europa y América. Durante este periodo, la economía basada en el trabajo manual fue reemplazada por otra dominada por la industria y la manufactura. Las innovaciones tecnológicas, como la máquina de vapor de James Watt, revolucionaron los métodos de producción y transporte.",

            "El ADN (ácido desoxirribonucleico) es una molécula compleja que contiene las instrucciones genéticas utilizadas en el desarrollo y funcionamiento de todos los organismos vivos conocidos. Su estructura de doble hélice fue descubierta por James Watson y Francis Crick en 1953, basándose en los datos cristalográficos de Rosalind Franklin. El ADN está compuesto por nucleótidos, cada uno conteniendo una base nitrogenada (adenina, guanina, citosina o timina), un grupo fosfato y un azúcar.",

            "La Primera Guerra Mundial (1914-1918) fue un conflicto bélico global centrado en Europa que comenzó el 28 de julio de 1914 y finalizó el 11 de noviembre de 1918. Esta guerra involucró a todas las grandes potencias del mundo, que se alinearon en dos bloques opuestos: los Aliados (Francia, Reino Unido, Rusia y más tarde Estados Unidos) y las Potencias Centrales (Alemania, Austria-Hungría, Imperio Otomano y Bulgaria).",

            "El cerebro humano es el órgano central del sistema nervioso, ubicado en el cráneo y protegido por el líquido cefalorraquídeo. Pesa aproximadamente 1.5 kilogramos y está dividido en diferentes regiones, cada una con funciones específicas. El cerebro procesa la información sensorial, controla los movimientos corporales, regula funciones fisiológicas y es responsable de funciones cognitivas superiores como el pensamiento, la memoria y el lenguaje.",

            "La teoría de la relatividad, desarrollada por Albert Einstein a principios del siglo XX, revolucionó nuestra comprensión del espacio, el tiempo y la gravedad. La relatividad especial, publicada en 1905, establece que las leyes de la física son las mismas para todos los observadores en movimiento uniforme, y que la velocidad de la luz en el vacío es constante independientemente del movimiento de la fuente o del observador. La relatividad general, publicada en 1915, describe la gravedad como una curvatura del espacio-tiempo causada por la masa y la energía.",

            "La Constitución Política de Colombia de 1991 es la carta magna de la República de Colombia. Fue promulgada el 4 de julio de 1991 y reemplazó a la Constitución de 1886. Esta nueva constitución estableció a Colombia como un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista. La constitución garantiza los derechos fundamentales, sociales, económicos y culturales de los ciudadanos colombianos.",

            "El Renacimiento fue un movimiento cultural que se desarrolló en Europa Occidental durante los siglos XV y XVI. Marcó la transición entre la Edad Media y la Edad Moderna, caracterizándose por un renovado interés en el pasado grecorromano clásico y un enfoque en el humanismo. Durante este periodo florecieron las artes, la arquitectura y la literatura, con figuras destacadas como Leonardo da Vinci, Miguel Ángel y Rafael en Italia, y El Greco y Cervantes en España.",

            "La inteligencia artificial (IA) es la simulación de procesos de inteligencia humana por parte de máquinas, especialmente sistemas informáticos. Estos procesos incluyen el aprendizaje (la adquisición de información y reglas para el uso de la información), el razonamiento (usando las reglas para llegar a conclusiones aproximadas o definitivas) y la autocorrección. La IA se aplica en campos tan diversos como la medicina, los vehículos autónomos, la traducción de idiomas y los asistentes virtuales.",

            "Madrid es la capital de España y una de las ciudades más grandes de Europa. Tiene una población de aproximadamente 3.3 millones de habitantes. Fundada en el siglo IX, Madrid se convirtió en la capital española en 1561. La ciudad alberga el Palacio Real, la plaza Mayor y el famoso museo del Prado, que contiene una de las colecciones de arte más importantes del mundo.",

            "El agua es una molécula compuesta por dos átomos de hidrógeno y uno de oxígeno (H2O). Es esencial para la vida en la Tierra y cubre aproximadamente el 71% de la superficie del planeta. El agua existe en tres estados: líquido, sólido (hielo) y gaseoso (vapor). El punto de ebullición del agua es de 100 grados Celsius a nivel del mar, mientras que su punto de congelación es de 0 grados Celsius.",

            "La fotosíntesis es un proceso que realizan las plantas para convertir la luz del sol en energía química. Este proceso produce oxígeno como subproducto. La fotosíntesis ocurre principalmente en las hojas de las plantas, donde la clorofila, un pigmento verde, absorbe la energía solar. Esta energía se utiliza para combinar dióxido de carbono y agua en glucosa, liberando oxígeno en el proceso.",

            "Miguel de Cervantes escribió Don Quijote de la Mancha, considerada la primera novela moderna. Fue publicada en dos partes, la primera en 1605 y la segunda en 1615. La obra narra las aventuras de Don Quijote, un hidalgo que enloquece tras leer demasiados libros de caballerías y decide convertirse en caballero andante. Es una de las obras más traducidas y adaptadas de la literatura mundial.",

            "La Revolución Industrial comenzó en Gran Bretaña a finales del siglo XVIII. Trajo cambios significativos en la agricultura, manufactura, minería y transporte. Una de las invenciones clave fue la máquina de vapor de James Watt, que permitió la mecanización de la producción. La Revolución Industrial marcó el inicio de la economía moderna y transformó radicalmente la sociedad, con la aparición de las fábricas y la urbanización.",

            "El Sol es una estrella en el centro de nuestro sistema solar. Proporciona luz y calor, lo que permite la vida en la Tierra. El Sol tiene aproximadamente 4,600 millones de años y se espera que permanezca estable durante otros 5,000 millones de años. Es una esfera de plasma caliente compuesta principalmente de hidrógeno y helio, y genera su energía a través de la fusión nuclear.",

            "Pablo Picasso fue un pintor y escultor español, creador del cubismo junto con Georges Braque. Es conocido por obras como 'Guernica' y 'Las señoritas de Avignon'. Nacido en Málaga en 1881, Picasso es considerado uno de los artistas más influyentes del siglo XX. Su obra abarca diferentes estilos, desde el realismo de sus primeros años hasta el cubismo y el surrealismo."
        ]

        # Multiplicamos los contextos para tener más ejemplos
        contexts = contexts * 5  # 85 contextos en total

        # Lista de patrones de preguntas para generar ejemplos diversos
        patterns = [
            "¿Qué es {}?",
            "¿Cuál es la importancia de {}?",
            "¿Quién descubrió {}?",
            "¿Dónde se desarrolló {}?",
            "¿Cuándo ocurrió {}?",
            "¿Por qué es importante {}?",
            "¿Cómo funciona {}?"
        ]

        examples = []

        for context in contexts:
            # Dividimos el contexto en frases
            sentences = re.split(r'(?<=[.!?])\s+', context)

            # Tomamos algunas frases como base para preguntas
            for j, sentence in enumerate(sentences[:3]):  # Hasta 3 preguntas por contexto
                if len(sentence) < 15:  # Ignoramos frases muy cortas
                    continue

                # Extraemos posibles respuestas (sustantivos o frases nominales)
                words = sentence.split()
                if len(words) < 3:
                    continue

                # Eligimos una parte de la frase como respuesta
                start_idx = random.randint(0, max(0, len(words) - 3))
                length = random.randint(1, min(3, len(words) - start_idx))
                answer = " ".join(words[start_idx:start_idx + length])

                # Generamos una pregunta usando uno de los patrones
                pattern = random.choice(patterns)

                # Extraemos un tema para la pregunta
                key_terms = re.findall(r'\b[A-Z][a-zA-Z]{3,}\b', context)
                if key_terms:
                    topic = random.choice(key_terms)
                else:
                    # Si no hay términos con mayúscula, usamos las primeras palabras
                    topic = " ".join(words[:2])

                question = pattern.format(topic)

                # Creamos el ejemplo
                examples.append({
                    'context': context,
                    'question': question,
                    'answer': answer
                })

        return examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        try:
            if isinstance(idx, list):
                # Si idx es una lista, solo tomamos el primer elemento
                # Esto soluciona el error "list indices must be integers or slices, not list"
                idx = idx[0] if len(idx) > 0 else 0

            example = self.examples[idx]

            context = example['context']
            question = example['question']
            answer = example['answer']

            # Acortamos el contexto si es muy largo
            max_context_length = min(1000, self.max_length // 2)
            if len(context) > max_context_length:
                context = context[:max_context_length]

            # Formato para modelos de lenguaje causal
            full_text = f"Contexto: {context}\nRespuesta: {answer}\nPregunta: {question}"

            # Tokenizamos el texto
            encodings = self.tokenizer(
                full_text,
                truncation=True,
                max_length=self.max_length,
                padding="max_length",
                return_tensors="pt"
            )

            # Preparamos las entradas y etiquetas
            input_ids = encodings["input_ids"].squeeze()
            attention_mask = encodings["attention_mask"].squeeze()

            # Para entrenamiento causal, las etiquetas son los input_ids con desplazamiento
            labels = input_ids.clone()

            # Enmascaramos las partes que no queremos predecir (contexto y respuesta)
            prompt_text = f"Contexto: {context}\nRespuesta: {answer}\nPregunta:"
            prompt_ids = self.tokenizer.encode(prompt_text, add_special_tokens=False)

            # Establecemos -100 para las partes enmascaradas (no contribuyen a la loss)
            prompt_len = min(len(prompt_ids), len(labels))
            labels[:prompt_len] = -100

            return {
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": labels
            }

        except Exception as e:
            print(f"Error al procesar ejemplo {idx}: {e}")
            # Devolvemos un tensor vacío como fallback
            return {
                "input_ids": torch.zeros(self.max_length, dtype=torch.long),
                "attention_mask": torch.zeros(self.max_length, dtype=torch.long),
                "labels": torch.zeros(self.max_length, dtype=torch.long)
            }

# Función para dividir el dataset en entrenamiento y validación
def split_dataset(dataset, test_size=0.2):
    indices = list(range(len(dataset)))
    random.shuffle(indices)
    split_idx = int(len(indices) * (1 - test_size))
    train_indices = indices[:split_idx]
    val_indices = indices[split_idx:]

    # Creamos subconjuntos que referencian al conjunto original
    class IndexSubset(Dataset):
        def __init__(self, dataset, indices):
            self.dataset = dataset
            self.indices = indices

        def __len__(self):
            return len(self.indices)

        def __getitem__(self, idx):
            # Aseguramos que idx sea un entero
            if isinstance(idx, list):
                idx = idx[0] if len(idx) > 0 else 0
            return self.dataset[self.indices[idx]]

    train_dataset = IndexSubset(dataset, train_indices)
    val_dataset = IndexSubset(dataset, val_indices)

    return train_dataset, val_dataset

# Función para cargar el modelo y tokenizer
def load_model_and_tokenizer(model_name, device):
    print(f"Cargando el tokenizer para {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Aseguramos que el tokenizer tenga un token de padding
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print(f"Cargando el modelo {model_name}...")
    try:
        # Intentamos cargar el modelo con FP16 para ahorrar memoria
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map='auto'
        )
    except Exception as e:
        print(f"Error al cargar con device_map='auto': {e}")
        print("Intentando cargar el modelo con configuración alternativa...")

        # Intentamos cargar con menos optimizaciones
        model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to(device)

    return model, tokenizer

# Función para entrenar el modelo
def train_model(model, tokenizer, train_dataloader, val_dataloader, num_epochs, lr,
                gradient_accumulation_steps, device, output_dir):
    # Configuramos el optimizador
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,
        weight_decay=0.01,
        eps=1e-6,
    )

    # Número total de pasos de entrenamiento
    num_update_steps_per_epoch = len(train_dataloader) // gradient_accumulation_steps
    max_train_steps = num_epochs * num_update_steps_per_epoch

    # Scheduler para el learning rate
    lr_scheduler = get_scheduler(
        name="cosine",
        optimizer=optimizer,
        num_warmup_steps=int(0.1 * max_train_steps),
        num_training_steps=max_train_steps,
    )

    # Inicializamos WandB para tracking
    wandb.init(project="gpt-neo-question-generation", name=f"gpt-neo-qa-generator")

    # Métricas para seguimiento
    train_losses = []
    val_losses = []
    best_val_loss = float("inf")

    print("\n== ENTRENAMIENTO DEL MODELO ==\n")

    for epoch in range(num_epochs):
        print(f"Iniciando época {epoch+1}/{num_epochs}")

        # Entrenamiento
        model.train()
        train_loss = 0.0
        train_steps = 0

        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))

        for step, batch in progress_bar:
            # Verificamos la forma de los tensores
            for k, v in batch.items():
                if v.dim() == 1:
                    batch[k] = v.unsqueeze(0)

            # Transferimos los tensores al dispositivo
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward pass
            outputs = model(**batch, use_cache=False)
            loss = outputs.loss

            # Normalizar la pérdida por el factor de acumulación
            loss = loss / gradient_accumulation_steps

            # Backward pass
            loss.backward()

            # Acumulamos la pérdida
            train_loss += loss.item() * gradient_accumulation_steps

            # Actualizamos los parámetros cada gradient_accumulation_steps
            if ((step + 1) % gradient_accumulation_steps == 0) or (step == len(train_dataloader) - 1):
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                train_steps += 1

            # Actualizamos la barra de progreso
            progress_bar.set_description(f"Época {epoch+1} - Loss: {loss.item() * gradient_accumulation_steps:.4f}")

        # Calculamos la pérdida promedio de entrenamiento
        avg_train_loss = train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss)

        # Evaluación
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Evaluando"):
                # Verificamos la forma de los tensores
                for k, v in batch.items():
                    if v.dim() == 1:
                        batch[k] = v.unsqueeze(0)

                # Transferimos los tensores al dispositivo
                batch = {k: v.to(device) for k, v in batch.items()}

                # Forward pass
                outputs = model(**batch, use_cache=False)

                # Acumulamos la pérdida
                val_loss += outputs.loss.item()

        # Calculamos la pérdida promedio de validación
        avg_val_loss = val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss)

        # Guardamos el mejor modelo
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            model_path = os.path.join(output_dir, "best_model")
            try:
                model.save_pretrained(model_path)
                tokenizer.save_pretrained(model_path)
                print(f"Mejor modelo guardado en {model_path}")
            except Exception as e:
                print(f"Error al guardar el mejor modelo: {e}")

        # Guardamos el checkpoint por época
        model_path = os.path.join(output_dir, f"epoch_{epoch+1}")
        try:
            model.save_pretrained(model_path)
            tokenizer.save_pretrained(model_path)
            print(f"Checkpoint de época {epoch+1} guardado en {model_path}")
        except Exception as e:
            print(f"Error al guardar checkpoint: {e}")

        # Logeamos métricas en WandB
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "learning_rate": lr_scheduler.get_last_lr()[0]
        })

        # Imprimimos el resumen de la época
        print(f"Época {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")

    # Guardamos el modelo final
    model_path = os.path.join(output_dir, "final_model")
    try:
        model.save_pretrained(model_path)
        tokenizer.save_pretrained(model_path)
        print(f"Modelo final guardado en {model_path}")
    except Exception as e:
        print(f"Error al guardar el modelo final: {e}")

    # Graficamos las pérdidas
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.savefig(os.path.join(output_dir, "loss_plot.png"))
    plt.close()

    wandb.finish()

    return model, tokenizer, train_losses, val_losses

def generate_question(context, answer, model, tokenizer, max_new_tokens=50):
    try:
        # Acortamos el contexto para evitar problemas de memoria
        if len(context) > 500:
            context = context[:500]

        # Simplificamos el prompt
        prompt = f"Contexto: {context}\nRespuesta: {answer}\nPregunta:"

        # Tokenizamos el prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Simplificamos los parámetros de generación
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=500,  # Usar max_length en lugar de max_new_tokens
            num_beams=3,     # Reducimos el número de beams
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,  # Aseguramos que el pad_token_id esté definido
            no_repeat_ngram_size=2
        )

        # Decodificamos la salida
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extraemos solo la parte de la pregunta
        if "Pregunta:" in generated_text:
            question = generated_text.split("Pregunta:")[1].strip()
            question = question.split(".")[0] + "." if "." in question else question
            return question
        else:
            return "¿Qué es " + answer + "?"

    except Exception as e:
        print(f"Error en la generación: {e}")
        # Proporcionamos una pregunta de fallback
        return f"¿Cuál es la importancia de {answer}?"

# Función para generar distractores
def generate_distractors(context, correct_answer, question, model, tokenizer, num_distractors=3):
    # Preparamos el prompt
    prompt = f"Contexto: {context}\nRespuesta correcta: {correct_answer}\nPregunta: {question}\nGenera {num_distractors} opciones incorrectas pero plausibles:"

    # Tokenizamos el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generamos los distractores usando max_new_tokens en lugar de max_length
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=120,  # Usamos max_new_tokens en lugar de max_length
        num_beams=8,
        temperature=0.7,
        top_p=0.9,
        top_k=40,
        do_sample=False,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # Decodificamos la salida
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraemos solo la parte de los distractores
    distractors_text = ""
    if "opciones incorrectas pero plausibles:" in generated_text:
        distractors_text = generated_text.split("opciones incorrectas pero plausibles:")[1].strip()

    # Procesamos los distractores generados
    distractors = []

    # Intentamos diferentes métodos para extraer los distractores
    # Método 1: División por saltos de línea
    if "\n" in distractors_text:
        for line in distractors_text.split("\n"):
            line = line.strip()
            if line and not line.isspace():
                # Limpiamos la línea de numeraciones
                clean_line = re.sub(r'^\d+[\.\)-]\s*', '', line).strip()
                if clean_line:
                    distractors.append(clean_line)

    # Método 2: Extracción mediante expresiones regulares
    if not distractors:
        pattern = r'\d+[\.\)-]\s*([^0-9\.\)-]+?)(?=\d+[\.\)-]|$)'
        matches = re.findall(pattern, distractors_text)
        if matches:
            distractors = [match.strip() for match in matches if match.strip()]

    # Si no hemos obtenido suficientes distractores, usamos el método de fallback
    if len(distractors) < num_distractors:
        # Extraemos entidades nombradas o frases del contexto que no sean la respuesta correcta
        sentences = re.split(r'(?<=[.!?])\s+', context)
        candidate_distractors = []

        for sentence in sentences:
            # Dividimos la frase en fragmentos
            fragments = re.split(r'[,;:]', sentence)
            for fragment in fragments:
                fragment = fragment.strip()
                if (fragment and len(fragment) > 3 and fragment != correct_answer
                        and not fragment.lower() in correct_answer.lower()
                        and not correct_answer.lower() in fragment.lower()):
                    candidate_distractors.append(fragment)

        # Seleccionamos fragmentos aleatorios si tenemos suficientes candidatos
        if candidate_distractors:
            random.shuffle(candidate_distractors)
            # Completamos los distractores necesarios
            needed = num_distractors - len(distractors)
            for i in range(min(needed, len(candidate_distractors))):
                # Limitamos la longitud para que no sean muy largos
                distractor = candidate_distractors[i]
                if len(distractor) > 50:
                    words = distractor.split()
                    distractor = " ".join(words[:7])  # Limitamos a 7 palabras
                distractors.append(distractor)

    # Si aún no tenemos suficientes, añadimos distractores genéricos
    generic_distractors = [
        "No se menciona en el contexto",
        "Información no disponible",
        "Datos insuficientes"
    ]

    while len(distractors) < num_distractors:
        for distractor in generic_distractors:
            if distractor not in distractors:
                distractors.append(distractor)
                if len(distractors) >= num_distractors:
                    break

    # Nos aseguramos de devolver exactamente el número solicitado
    return distractors[:num_distractors]

# Función para probar el modelo con ejemplos
def test_model_with_examples(model, tokenizer, num_examples=3):
    # Ejemplos de textos y respuestas para probar
    test_examples = [
        {
            "context": "Los planetas del sistema solar en orden desde el Sol son: Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno. Plutón fue considerado un planeta hasta 2006, cuando fue reclasificado como planeta enano.",
            "answer": "Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno"
        },
        {
            "context": "El ADN (ácido desoxirribonucleico) es una molécula compleja que contiene las instrucciones genéticas utilizadas en el desarrollo y funcionamiento de todos los organismos vivos. La estructura del ADN fue descubierta en 1953 por James Watson y Francis Crick.",
            "answer": "James Watson y Francis Crick"
        },
        {
            "context": "La fotosíntesis es un proceso utilizado por las plantas y otros organismos para convertir la energía lumínica en energía química. Durante este proceso, el dióxido de carbono y el agua se combinan para formar glucosa y oxígeno utilizando la energía de la luz solar.",
            "answer": "energía lumínica en energía química"
        }
    ]

    print("\n== GENERANDO PREGUNTAS DE OPCIÓN MÚLTIPLE ==\n")

    generated_questions = []

    for example in test_examples[:num_examples]:
        try:
            context = example["context"]
            answer = example["answer"]

            print(f"CONTEXTO: {textwrap.fill(context, width=80)}")
            print(f"RESPUESTA CORRECTA: {answer}")

            # Generamos la pregunta con manejo de errores
            question = generate_question(context, answer, model, tokenizer)
            print(f"PREGUNTA GENERADA: {question}")

            # Si llegamos aquí sin errores, generamos los distractores
            try:
                distractores = generate_distractors(context, answer, question, model, tokenizer)
            except Exception as e:
                print(f"Error al generar distractores: {e}")
                # Generamos distractores de fallback
                distractores = [
                    "Información no disponible en el contexto",
                    "Datos insuficientes",
                    "No se menciona en el texto"
                ]

            # Mezclamos las opciones
            options = [answer] + distractores
            random.shuffle(options)

            # Identificamos el índice de la respuesta correcta
            correct_index = options.index(answer)

            # Mostramos las opciones
            print("OPCIONES:")
            for i, option in enumerate(options):
                print(f"{i+1}. {option}" + (" (CORRECTA)" if i == correct_index else ""))

            # Guardamos la pregunta generada
            mc_question = {
                "context": context,
                "question": question,
                "options": options,
                "correct_option_index": correct_index
            }
            generated_questions.append(mc_question)

            print("\n" + "="*80 + "\n")

        except Exception as e:
            print(f"Error al procesar ejemplo: {e}")
            print("\n" + "="*80 + "\n")
            continue

    # Guardamos las preguntas generadas si hay alguna
    if generated_questions:
        output_path = os.path.join(OUTPUT_DIR, "generated_questions.json")
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(generated_questions, f, ensure_ascii=False, indent=4)
        print(f"Preguntas generadas guardadas en {output_path}")

    return generated_questions

# Función principal
def main():
    # Iniciar wandb
    try:
        wandb.login()
    except Exception as e:
        print(f"Error al iniciar wandb: {e}")
        print("Continuando sin tracking de wandb")

    # Verificamos si existe el archivo XQuAD
    xquad_path = "xquad.es.json"
    use_synthetic = not os.path.exists(xquad_path)

    if use_synthetic:
        print("No se encontró el archivo XQuAD. Se utilizará el dataset sintético.")
    else:
        print(f"Se encontró el archivo XQuAD en {xquad_path}. Se utilizará este dataset.")

    # Cargar modelo y tokenizer
    try:
        model, tokenizer = load_model_and_tokenizer(MODEL_NAME, DEVICE)
    except Exception as e:
        print(f"Error al cargar el modelo completo: {e}")
        print("Intentando con un modelo más pequeño...")

        # Intentamos con un modelo más pequeño si hay problemas de memoria
        MODEL_NAME_SMALL = "EleutherAI/gpt-neo-125M"
        model, tokenizer = load_model_and_tokenizer(MODEL_NAME_SMALL, DEVICE)

    # Crear dataset
    full_dataset = XQuADDataset(
        file_path=xquad_path if not use_synthetic else None,
        use_synthetic=use_synthetic,
        tokenizer=tokenizer,
        max_length=MAX_LENGTH
    )

    # Dividir en train y validation
    train_dataset, val_dataset = split_dataset(full_dataset)

    print(f"Dataset dividido: {len(train_dataset)} ejemplos de entrenamiento, {len(val_dataset)} de validación")

    # Crear dataloaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False
    )

    # Entrenar el modelo
    model, tokenizer, train_losses, val_losses = train_model(
        model=model,
        tokenizer=tokenizer,  # Añadimos el tokenizer como parámetro
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        num_epochs=NUM_EPOCHS,
        lr=LEARNING_RATE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        device=DEVICE,
        output_dir=OUTPUT_DIR
    )

    # Liberar memoria
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Después del entrenamiento, cargamos un modelo más pequeño para la generación
    print("Cargando un modelo más pequeño para la generación...")
    try:
        generation_model = AutoModelForCausalLM.from_pretrained(
            "EleutherAI/gpt-neo-125M",  # Usamos el modelo más pequeño
            torch_dtype=torch.float32    # Evitamos usar float16 para prevenir errores
        ).to(DEVICE)

        # Probar el modelo generando preguntas
        test_model_with_examples(generation_model, tokenizer)

    except Exception as e:
        print(f"Error al cargar el modelo de generación: {e}")
        print("Intento final con un modelo aún más simple...")

        # Si todavía falla, usamos el modelo distilgpt2
        try:
            from transformers import GPT2LMHeadModel
            generation_model = GPT2LMHeadModel.from_pretrained("distilgpt2").to(DEVICE)
            test_model_with_examples(generation_model, tokenizer)
        except Exception as e:
            print(f"Error final: {e}")
            print("No se pudieron generar preguntas.")

    print("\n¡Proceso completado!")

# Ejecutamos el código principal
if __name__ == "__main__":
    main()

Utilizando device: cuda

===== INSTRUCCIONES PARA DESCARGAR DATOS =====

1. Descarga XQuAD-ES desde GitHub:
   - Ve a: https://github.com/deepmind/xquad
   - Descarga el archivo 'xquad.es.json' desde la carpeta 'data'

2. Sube el archivo a tu sesión de Colab:
   - Usa el menú a la izquierda -> Archivos -> Subir
   - Selecciona el archivo xquad.es.json que descargaste

3. Ejecuta este notebook después de subir el archivo

Si tienes problemas para descargar XQuAD, puedes continuar con el dataset sintético.



<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: est-juand-quiroga (est-juand-quiroga-universidad-militar-nueva-granada) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

PREGUNTA GENERADA: ¿Qué está pasando?

Agradezco la sugerencia de que este tipo de información no es una solución para el problema de estas cosas, sino para algunos estudios.
OPCIONES:
1. ¿Cómo puede hacer esto? ¿Por qué?
2. Mercurio, Venus, Tierra, Marte, Júpiter, Saturno, Urano y Neptuno (CORRECTA)
3. Esto es lo que hacemos hoy en día.
4. Esta es la primera vez que estamos haciendo esto.


CONTEXTO: El ADN (ácido desoxirribonucleico) es una molécula compleja que contiene las
instrucciones genéticas utilizadas en el desarrollo y funcionamiento de todos
los organismos vivos. La estructura del ADN fue descubierta en 1953 por James
Watson y Francis Crick.
RESPUESTA CORRECTA: James Watson y Francis Crick


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


PREGUNTA GENERADA: ¿Cómo puede ayudar a hacer esto?

Agradezco la ayuda de la Comisión de Medio Ambiente (CMA) y de los Estados Unidos (Usuarios) para que el estudio de este proyecto se realicara en la última década.
OPCIONES:
1. James Watson y Francis Crick (CORRECTA)
2. En el caso de que el ADN pueda ser usado, el único objeto de estas estructuras está en la dirección de la base de datos. Por ejemplo, los datos de los únicos estudiantes que están haciendo esto podrían ser usados.
3. El objetivo de
4. El ADN debe ser utilizado en el ámbito de la estructuración de los estudios de esta CMA.


CONTEXTO: La fotosíntesis es un proceso utilizado por las plantas y otros organismos para
convertir la energía lumínica en energía química. Durante este proceso, el
dióxido de carbono y el agua se combinan para formar glucosa y oxígeno
utilizando la energía de la luz solar.
RESPUESTA CORRECTA: energía lumínica en energía química


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


PREGUNTA GENERADA: ¿Por qué es posible mantener el cuerpo de los científicos?

Aquí está el primer proyecto de energia que incluye el establecimiento del estudio.
OPCIONES:
1. El agua
2. energía lumínica en energía química (CORRECTA)
3. El estado de energía
4. El carbono


Preguntas generadas guardadas en gpt-neo-question-generator/generated_questions.json

¡Proceso completado!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
